In [5]:
from garminconnect import Garmin
import fitdecode
import zipfile
import io

In [6]:
client = Garmin("ju.roulle@gmail.com", "Julien35830")
client.login()
activities = client.get_activities(0, 1)

In [7]:
activity = activities[0]
activity_id = activity["activityId"]

In [4]:
data = client.download_activity(activity_id, dl_fmt=client.ActivityDownloadFormat.ORIGINAL)

with zipfile.ZipFile(io.BytesIO(data), "r") as zip_file:

    for file_name in zip_file.namelist():
        file_content = zip_file.read(file_name)
        
        # Extract all files in the zip archive into the memory file
        memory_file = io.BytesIO()

        # Read the contents of the extracted file into an io.BytesIO buffer
        memory_file.write(file_content)

        # Reset the memory file's position to the beginning
        memory_file.seek(0)
        with fitdecode.FitReader(memory_file) as fit_file:
            for frame in fit_file:
                if frame.frame_type == fitdecode.FIT_FRAME_DATA:
                    print(frame.name)

file_id
file_creator
unknown_288
unknown_327
unknown_326
unknown_327
unknown_326
event
device_info
device_info
device_info
device_info
device_info
device_info
unknown_22
unknown_141
device_settings
user_profile
unknown_147
unknown_147
unknown_147
unknown_147
unknown_147
unknown_79
sport
unknown_13
zones_target
record
unknown_233
gps_metadata
record
unknown_233
gps_metadata
record
unknown_233
gps_metadata
record
unknown_233
gps_metadata
record
unknown_233
gps_metadata
record
unknown_233
gps_metadata
record
unknown_233
gps_metadata
record
unknown_233
gps_metadata
record
unknown_233
gps_metadata
record
unknown_233
gps_metadata
record
unknown_233
gps_metadata
record
unknown_233
gps_metadata
unknown_325
record
unknown_233
gps_metadata
record
unknown_233
gps_metadata
record
unknown_233
gps_metadata
record
unknown_233
gps_metadata
record
unknown_233
gps_metadata
record
unknown_233
gps_metadata
record
unknown_233
gps_metadata
record
unknown_233
gps_metadata
record
unknown_233
gps_metadata
reco

In [7]:
from stridze.db import engine
from sqlalchemy import MetaData
metadata = MetaData()
metadata.reflect(engine)
for table in reversed(metadata.sorted_tables):
    print(table.name)
    if table.name == 'user':
        table.drop(engine)

records
laps
activities
users
user
strava


In [4]:
from sqlmodel import BigInteger, Column, Field, Relationship, SQLModel
from stridze.db import engine
from typing import List, Optional
from datetime import datetime

class Activity(SQLModel, table=True):
    id: int = Field(default=None, primary_key=True, index=True)
    elapsed_time: int = Field(nullable=True)
    moving_time: int = Field(nullable=True)
    distance: int = Field(nullable=True)
    elevation_gain: int = Field(nullable=True)
    elevation_loss: int = Field(nullable=True)
    average_pace: int = Field(nullable=True)
    average_moving_pace: int = Field(nullable=True)
    average_cadence: int = Field(nullable=True)
    average_heart_rate: int = Field(nullable=True)
    max_heart_rate: int = Field(nullable=True)
    average_stride_length: float = Field(nullable=True, precision=8, asdecimal=True)
    average_temperature: float = Field(nullable=True, precision=8, asdecimal=True)
    calories: int = Field(nullable=True)

    user_id: int = Field(foreign_key="users.id")
    user: Relationship = Relationship(back_populates="activities")
    points: Relationship = Relationship(back_populates="activity")
    laps: Relationship = Relationship(back_populates="activity")
    tcx_laps: Relationship = Relationship(back_populates="activity")
    records: Relationship = Relationship(back_populates="activity")

class FitActivity(SQLModel, table=True, extend_existing=True):
    id: int = Field(default=None, primary_key=True, index=True)
    garmin_id: Optional[int] = Field(sa_column=Column(BigInteger()))

    laps: List["Lap"] = Relationship(back_populates="fitactivity")
    records: List["Record"] = Relationship(back_populates="fitactivity")

class Lap(SQLModel, table=True, extend_existing=True):
    id: int = Field(default=None, primary_key=True, index=True)
    timestamp: Optional[datetime]
    start_time: Optional[datetime]
    start_position_lat: Optional[int]
    start_position_long: Optional[int]
    end_position_lat: Optional[int]
    end_position_long: Optional[int]
    total_elapsed_time: Optional[float]
    total_timer_time: Optional[float]
    total_distance: Optional[float]
    total_strides: Optional[int]
    total_work: Optional[float]
    time_standing: Optional[float]
    avg_left_power_phase: Optional[str]
    avg_left_power_phase_peak: Optional[str]
    avg_right_power_phase: Optional[str]
    avg_right_power_phase_peak: Optional[str]
    avg_power_position: Optional[str]
    max_power_position: Optional[str]
    enhanced_avg_speed: Optional[float]
    enhanced_max_speed: Optional[float]
    enhanced_avg_altitude: Optional[float]
    enhanced_min_altitude: Optional[float]
    enhanced_max_altitude: Optional[float]
    total_grit: Optional[float]
    avg_flow: Optional[float]
    message_index: Optional[int]
    total_calories: Optional[int]
    total_fat_calories: Optional[float]
    avg_speed: Optional[float]
    max_speed: Optional[float]
    avg_power: Optional[float]
    max_power: Optional[float]
    total_ascent: Optional[int]
    total_descent: Optional[int]
    num_lengths: Optional[int]
    normalized_power: Optional[float]
    left_right_balance: Optional[float]
    first_length_index: Optional[int]
    avg_stroke_distance: Optional[float]
    num_active_lengths: Optional[int]
    wkt_step_index: Optional[int]
    avg_vertical_oscillation: Optional[float]
    avg_stance_time_percent: Optional[float]
    avg_stance_time: Optional[float]
    stand_count: Optional[int]
    avg_vertical_ratio: Optional[float]
    avg_stance_time_balance: Optional[float]
    avg_step_length: Optional[float]
    event: Optional[str]
    event_type: Optional[str]
    avg_heart_rate: Optional[int]
    max_heart_rate: Optional[int]
    avg_running_cadence: Optional[int]
    max_running_cadence: Optional[int]
    intensity: Optional[float]
    lap_trigger: Optional[str]
    sport: Optional[str]
    event_group: Optional[int]
    swim_stroke: Optional[str]
    sub_sport: Optional[str]
    avg_temperature: Optional[int]
    max_temperature: Optional[int]
    avg_fractional_cadence: Optional[float]
    max_fractional_cadence: Optional[float]
    total_fractional_cycles: Optional[float]
    avg_left_torque_effectiveness: Optional[float]
    avg_right_torque_effectiveness: Optional[float]
    avg_left_pedal_smoothness: Optional[float]
    avg_right_pedal_smoothness: Optional[float]
    avg_combined_pedal_smoothness: Optional[float]
    avg_left_pco: Optional[float]
    avg_right_pco: Optional[float]
    avg_cadence_position: Optional[str]
    max_cadence_position: Optional[str]
    total_fractional_ascent: Optional[float]
    total_fractional_descent: Optional[float]

    fitactivity_id: Optional[int] = Field(default=None, foreign_key="fitactivity.id")
    fitactivity: Optional[FitActivity] = Relationship(back_populates="laps")

    records: List["Record"] = Relationship(back_populates="lap")

class Record(SQLModel, table=True, extend_existing=True):
    id: int = Field(default=None, primary_key=True, index=True)
    timestamp: Optional[datetime]
    position_lat: Optional[int]
    position_long: Optional[int]
    distance: Optional[float]
    enhanced_speed: Optional[float]
    enhanced_altitude: Optional[float]
    vertical_oscillation: Optional[float]
    stance_time_percent: Optional[float]
    stance_time: Optional[float]
    vertical_ratio: Optional[float]
    stance_time_balance: Optional[float]
    step_length: Optional[float]
    heart_rate: Optional[int]
    cadence: Optional[int]
    temperature: Optional[int]
    activity_type: Optional[str]
    fractional_cadence: Optional[float]

    lap_id: Optional[int] = Field(default=None, foreign_key="lap.id")
    lap: Optional[Lap] = Relationship(back_populates="records")

    fitactivity_id: Optional[int] = Field(default=None, foreign_key="fitactivity.id")
    fitactivity: Optional[FitActivity] = Relationship(back_populates="records")


In [1]:
from stridze.db import get_session
from stridze.db.models import Activity

session = get_session()
a = session.query(Activity).first()

In [2]:
a

Activity(distance=34448.5703125, average_heart_rate=125, duration=5210.9541015625, max_heart_rate=144, id=1, elapsed_time=5210, calories=711, garmin_id=12342269889, moving_time=5169, user_id=None, activity_type='cycling', elevation_gain=216, start_time=datetime.datetime(2023, 10, 16, 16, 56, 12), elevation_loss=211, start_position_lat=48.12922538258135, average_speed=6, start_position_long=-1.6958808805793524, max_speed=12)

In [3]:
a.activity_type

'cycling'

In [8]:
activity

{'activityId': 12342269889,
 'activityName': 'Rennes Cyclisme',
 'description': None,
 'startTimeLocal': '2023-10-16 16:56:12',
 'startTimeGMT': '2023-10-16 14:56:12',
 'activityType': {'typeId': 2,
  'typeKey': 'cycling',
  'parentTypeId': 17,
  'isHidden': False,
  'restricted': False,
  'trimmable': True},
 'eventType': {'typeId': 9, 'typeKey': 'uncategorized', 'sortOrder': 10},
 'comments': None,
 'parentId': None,
 'distance': 34448.5703125,
 'duration': 5210.9541015625,
 'elapsedDuration': 5210.9541015625,
 'movingDuration': 5169.1510009765625,
 'elevationGain': 216.7699999809265,
 'elevationLoss': 211.49000000953674,
 'averageSpeed': 6.611000061035157,
 'maxSpeed': 12.29800033569336,
 'startLatitude': 48.12922538258135,
 'startLongitude': -1.6958808805793524,
 'hasPolyline': True,
 'ownerId': 73254434,
 'ownerDisplayName': '1c2bd72d-dab7-430c-9eb0-88f0fcc8c9e8',
 'ownerFullName': 'Julien Roullé',
 'ownerProfileImageUrlSmall': 'https://s3.amazonaws.com/garmin-connect-prod/profile

In [9]:
import pandas as pd
from stridze.db import engine

from stridze.db.models import User

from sqlmodel import Session, select

session = Session(engine)

users = session.exec(select(User)).all()